1) make finetuning with kws

2) make eval with giving title, kws, and pros/cons??

we have

- keywords for positive and for negative sides of every hotel

- 1 var - GPT2 finetuning




now: 2 var - GPT2 finetuning with keywords



One issue with text generation is the lack of control in the direction it takes. 

In [5]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/gdrive')




Mounted at /gdrive


In [2]:
!unzip /gdrive/My\ Drive/nnproj/data1.zip

Archive:  /gdrive/My Drive/nnproj/data1.zip
  inflating: reviews_gpt.csv         
  inflating: reviews.csv             


In [3]:
reviews_gpt = pd.read_csv("reviews_gpt.csv", sep='\t')

In [ ]:
reviews_gpt.shape

(14361, 30)

In [4]:
reviews_gpt.head(1)

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites,reviews.dateAdded,is_full,lang,reviews.text.tokens,reviews.length
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,...,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com,NaN,True,en,"['Our', 'experience', 'at', 'Rancho', 'Valenci...",37


In [5]:
reviews_gpt[reviews_gpt['id'].isna()]

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,...,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites,reviews.dateAdded,is_full,lang,reviews.text.tokens,reviews.length


data changes. rating to 3 class

In [ ]:
reviews_gpt["reviews.rating"].value_counts()

In [ ]:
reviews_gpt["reviews.rating"][reviews_gpt["reviews.rating"] < 3] = 0 # neg
reviews_gpt["reviews.rating"][reviews_gpt["reviews.rating"] >= 4.5] = 1 # pos
reviews_gpt["reviews.rating"][(reviews_gpt["reviews.rating"] != 1) & (reviews_gpt["reviews.rating"] != 0)] = 2 # else

In [8]:
reviews_gpt["reviews.rating"].value_counts()

1.0    6928
2.0    5852
0.0    1581
Name: reviews.rating, dtype: int64

In [9]:
reviews_gpt.columns

Index(['id', 'dateAdded', 'dateUpdated', 'address', 'categories',
       'primaryCategories', 'city', 'country', 'keys', 'latitude', 'longitude',
       'name', 'postalCode', 'province', 'reviews.date', 'reviews.dateSeen',
       'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title',
       'reviews.userCity', 'reviews.userProvince', 'reviews.username',
       'sourceURLs', 'websites', 'reviews.dateAdded', 'is_full', 'lang',
       'reviews.text.tokens', 'reviews.length'],
      dtype='object')

In [10]:
reviews_gpt["reviews.rating"].unique()

array([1., 0., 2.])

In [14]:
reviews_gpt.to_csv("/gdrive/My Drive/nnproj/normal_reviews_sent_to3class.tsv", sep='\t')

gpt2 finetuning

In [2]:
! pip install transformers > /dev/null

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

# CREATE TEST SET

In [15]:
df = reviews_gpt[["id", "reviews.text", "name", "reviews.rating"]]

In [16]:
df.head(1)

,id,reviews.text,name,reviews.rating
0,AVwc252WIN2L1WUfpqLP,Our experience at Rancho Valencia was absolute...,Rancho Valencia Resort Spa,1.0


## EVAL

надо выбирать для теста те у кого точно есть id отеля чтоб на эвалюации были ключевые слова

In [17]:
df.query('id != ""').shape # only rows with non NAN ids of hotels

(14361, 4)

Create a very small test set to compare generated text with the reality


mind class balance and shuffle

In [19]:
g = df.query("id != ''").groupby('reviews.rating') # NON-NAN IDS SO WE COULD GET KWS FOR THE HOTEL
# g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))

test_set = pd.DataFrame(g.apply(lambda x: x.sample(n=67).reset_index(drop=True)))

In [20]:
test_set.shape

(201, 4)

In [21]:
test_set = test_set.reset_index(drop=True)

In [22]:
test_set["reviews.rating"].value_counts()

0.0    67
1.0    67
2.0    67
Name: reviews.rating, dtype: int64

In [23]:
from sklearn.utils import shuffle
test_set = shuffle(test_set)

In [24]:
test_set.head()

,id,reviews.text,name,reviews.rating
110,AWE778BPRxPSIh2RsmyR,Excelente! Fui por 8 dias y 7 noches. Un servi...,Days Inn Baltimore South/Glen Burnie,1.0
126,AVwd1buK_7pvs4fz9r3n,Perfect stay. I really enjoyed my time here. I...,Jayhawk Motel,1.0
119,AVwctAJvkufWRAb5yHGn,"I stayed in 809, and it was an absolute blast....",FOUND:RE Phoenix,1.0
124,AVwd4TMv_7pvs4fz-Ers,Small town on the state route - unexpected del...,Silver Sands Oceanfront Motel,1.0
180,AWEw9jFgIxWefVJwyBzX,We were for a week with our eleven month baby ...,DoubleTree by Hilton Hotel Washington DC - Sil...,2.0


In [25]:
df = df.loc[~df.index.isin(test_set.index)]

#Reset the indexes
test_set = test_set.reset_index()
df = df.reset_index()

In [26]:
#For the test set only, keep last 18 words in a new column, then remove them from original column
test_set['true_end'] = test_set['reviews.text'].str.split().str[-18:].apply(' '.join)
# оставляем только первые 18 слов (MIN 20)
test_set['reviews.text'] = test_set['reviews.text'].str.split().str[:-18].apply(' '.join) 


In [28]:
test_set.head(2)

,index,id,reviews.text,name,reviews.rating,true_end
0,110,AWE778BPRxPSIh2RsmyR,Excelente! Fui por 8 dias y 7 noches. Un servi...,Days Inn Baltimore South/Glen Burnie,1.0,te lleva hasta el downtown de Baltimore. Y a l...
1,126,AVwd1buK_7pvs4fz9r3n,Perfect stay. I really enjoyed my time here. I...,Jayhawk Motel,1.0,grocery and convenience stores and laundry all...


In [29]:
test_set.to_csv("/gdrive/My Drive/nnproj/normal_test_set.tsv", sep='\t')

dataset

# TRAIN

In [6]:
import json

In [7]:
with open("/gdrive/My Drive/nnproj/all_keywords_with_top20_with_default.json") as f:
    mykws = json.loads(f.read())

## kws with default

## get kws for hotel id

In [8]:
def get_train_kws(myid, sent):
    sent = int(sent)
    the_id = myid
    if not myid:
        the_id = "default"
    
    try:
        mykws[the_id]
    except:
        print("ID NOT FOUND IN KWS", the_id)
        the_id = "default"
        
    
    if sent == 0:
        s = "neg"
    elif sent == 1:
        s = "pos"
    elif sent == 2:
        return mykws[the_id]["pos"] + mykws[the_id]["neg"]
    else:
        raise Exception("unknown class for rating")
    
    if not mykws[the_id][s]:
        return mykws["default"][s]
    
    # normal case
    return mykws[the_id][s]


# DATASET

In [9]:
def join_keywords(hotel_id, sent, randomize=True):
    keywords = get_train_kws(hotel_id, sent)
    N = len(keywords)

    #random sampling and shuffle
    if randomize: 
        M = random.choice(range(N+1))
        keywords = keywords[:M]
        random.shuffle(keywords)

    return ','.join(keywords)

In [35]:
train_texts = list(df['reviews.text'])
train_names = list(df['name'])
train_ids = list(df['id'])
train_rating = list(df['reviews.rating'])

In [36]:
class ReviewsDataset(Dataset):  
    def __init__(self, texts, names, ids, rating, truncate=False, gpt2_type="gpt2", max_length=1024, randomize=True):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.texts = []

        for i in range(len(texts)):
          kw = join_keywords(ids[i], rating[i], randomize)
          self.texts.append(torch.tensor(
                self.tokenizer.encode(f"<|BOS|>{names[i]}<|SEP|>{kw}<|SEP|>{texts[i][:max_length]}<|EOS|>")
            ))
        if truncate:
            self.texts = self.texts[:20000]
        self.texts_count = len(self.texts)
    
    

    def __len__(self):
        return self.texts_count

    def __getitem__(self, item):
        
        return self.texts[item]


In [37]:
dataset = ReviewsDataset(train_texts, train_names, train_ids, train_rating, truncate=True, gpt2_type="gpt2")

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [39]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [11]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [12]:
import os
from tqdm import tqdm

In [26]:
device=torch.device("cuda")
model = model.cuda()

In [42]:
def train(
    dataset, model, tokenizer,
    batch_size=32, epochs=3, lr=2e-5,
    max_seq_len=300, warmup_steps=200,
    gpt2_type="gpt2", output_dir="/gdrive/My Drive/nnproj/normres/", output_prefix="reviews",
    test_mode=False,save_model_on_epoch=True,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [44]:
model = train(dataset, model, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Training epoch 0
0


14160it [20:06, 11.74it/s]


Training epoch 1
tensor(2.8803, device='cuda:0', grad_fn=<NllLossBackward0>)


14160it [20:02, 11.78it/s]


Training epoch 2
tensor(2.5943, device='cuda:0', grad_fn=<NllLossBackward0>)


14160it [20:07, 11.73it/s]


______

# load model

In [13]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("/gdrive/My Drive/nnproj/normres/reviews-2.pt"))

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

<All keys matched successfully>

test dataset

In [45]:
test_texts = list(test_set['reviews.text'])
test_names = list(test_set['name'])
test_ids = list(test_set['id'])

test_rating = list(test_set['reviews.rating'])
test_rating = list(map(int, test_rating))

In [46]:
len(test_rating)#, len(train_rating)

200

# generation

In [58]:
test_names[0], test_rating[0], test_ids[0], test_texts[0]

('Jayhawk Motel',
 1,
 'AVwd1buK_7pvs4fz9r3n',
 "Perfect stay. I really enjoyed my time here. It is close to the grey hound bus stop. It is quiet. For the price, it is an amazing place. 40.00 is great considering it's close to downtown and ybor city. Thete is")

In [59]:
keywords = join_keywords(test_ids[0], test_rating[0], randomize=False)

In [60]:
keywords

'an amazing place'

In [27]:
device = torch.device("cuda")
model.eval();

In [30]:
def populate(testtexts, testnames, testids, testrating):
    for i in range(len(testtexts)):
        if len(str(testtexts[i])) >= 1024:
            final.append("")
            continue
        ks = join_keywords(testids[i], testrating[i], randomize=False)

        prompt = "<|BOS|>" + str(testnames[i]) + "<|SEP|>" + str(ks) + "<|SEP|>" + str(testtexts[i])
        generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
        
        generated = generated.to(device)

        sample_outputs = model.generate(generated, 
                                  do_sample=True,   
                                  min_length=20, 
                                  max_length=768, #
                                  top_k=30,                                 
                                  top_p=0.7,        
                                  temperature=0.5,
                                  repetition_penalty=2.0,
                                  num_return_sequences=1
                                  )

        for i, sample_output in enumerate(sample_outputs):
            text = tokenizer.decode(sample_output, skip_special_tokens=True)
          
        final.append(text)


# add new cols

In [15]:
test_set = pd.read_csv("/gdrive/My Drive/nnproj/normal_test_set.tsv", sep='\t')

In [18]:
test_set.shape

(200, 7)

In [33]:
final = []

In [ ]:
populate(test_texts, test_names, test_ids, test_rating)

In [39]:
res = final

In [40]:
len(res)

200

In [42]:
test_set["GENERATED"] = res

In [50]:
results = []

In [51]:
for el, sep in zip(res, test_texts):
  if not el:
    results.append("")
    continue

  sep = str(sep)
  try:
    t = el.split(sep)[1]
  except:
    length = len(sep)
    t = el.split("<|SEP|>", maxsplit=2)[2]
    t = t[length:]
   
  results.append(t)
  

In [52]:
len(results)

200

In [53]:
test_set["generated_end"] = results

In [61]:
test_set = test_set[test_set["generated_end"] != ""]

In [64]:
test_set.shape

(188, 9)

In [66]:
test_set.to_csv("/gdrive/My Drive/nnproj/normal_res_with_generated_and_gen_end.tsv", sep='\t')

# konetsc